In [1]:
# -*- coding: utf-8 -*-
"""
    Types
"""
import functools

from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base

import dataclasses as dc
import datetime
import decimal
import typing

import dbcontroller as dbc

# URL
DATABASE_URL = "sqlite:///example.db"

# Base
Base = declarative_base()

# Manager
SQL = functools.partial(dbc.SQL, DATABASE_URL)

# Model
model = dbc.Model(sql=Base)


# DateTime Functions
class Date:
    datetime = lambda: datetime.datetime.now()
    date = lambda: datetime.date.today()
    time = lambda: datetime.datetime.now().time()


# Create your <types> here.
@model.sql
class Product:
    name: str
    aliases: list[str] | None = None
    stock: int | None = None
    is_available: bool | None = None
    created_on: datetime.datetime = dc.field(default_factory=Date.datetime)
    available_from: datetime.date = dc.field(default_factory=Date.date)
    same_day_shipping_before: datetime.time = dc.field(default_factory=Date.time)
    price: decimal.Decimal | None = None
    notes: list[dbc.Text] = dc.field(default_factory=list)
    is_object: dbc.JSON = dc.field(default_factory=dict)

    async def category(self) -> "Category":
        return Category(name="awesome")


@model.sql
class Category:
    name: str


In [2]:
# Regular Engine
engine = create_engine(DATABASE_URL, echo=True)

# Register Tables & Load -> Lazy-Loaded Tables (aka: Types)
dbc.Admin.register([Product, Category])
dbc.Admin.load()

# Create Tables
Base.metadata.create_all(engine)

2022-08-24 22:48:19,213 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-08-24 22:48:19,213 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("main_category")
2022-08-24 22:48:19,214 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-24 22:48:19,215 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("main_category")
2022-08-24 22:48:19,216 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-24 22:48:19,218 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("main_product")
2022-08-24 22:48:19,218 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-24 22:48:19,220 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("main_product")
2022-08-24 22:48:19,221 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-24 22:48:19,222 INFO sqlalchemy.engine.Engine 
CREATE TABLE main_category (
	_id INTEGER NOT NULL, 
	name VARCHAR(255), 
	PRIMARY KEY (_id)
)


2022-08-24 22:48:19,223 INFO sqlalchemy.engine.Engine [no key 0.00060s] ()
2022-08-24 22:48:19,227 INFO sqlalchemy.engine.Engine 
CREAT